# FIRST : IMPORT LIBRARIES 

In [1]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [48]:
#libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from  sklearn.metrics import classification_report, confusion_matrix
from sklearn import datasets, linear_model, metrics
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
import autocorrect
#from spellchecker import SpellChecker
import torch
#from transformers import BertTokenizer, BertModel
%matplotlib inline

In [3]:
df= pd.read_csv('IMDB Dataset.csv')

In [4]:
df .info

<bound method DataFrame.info of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


# SECOND : PRE-PROCESSING & DATA CLEANING 

In [7]:
df['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

#### so since the they are equal and not imbalanced towards one sentiment so no need for undersampling or oversampling

## Lowercasing 

 we apply lowercasing to ensure equal consideration for all letters 

In [8]:
def lowercase_text(text):
    return text.lower()

In [9]:
df['review'] = df['review'].apply(lowercase_text)

In [10]:
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


## Split the dataframe into feature and target dataframes 

In [11]:
dfr= df['review']
dfs = df['sentiment']

In [12]:
label_encoder = LabelEncoder()
dfs = label_encoder.fit_transform(dfs)

In [13]:
dfs 

array([1, 1, 1, ..., 0, 0, 0])

In [14]:
nltk.download('punkt')  # Download the Punkt tokenizer models

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SamaG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
dfr

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

In [16]:
tokenizer=ToktokTokenizer()

In [17]:
dfr.describe()

count                                                 50000
unique                                                49582
top       loved today's show!!! it was a variety and not...
freq                                                      5
Name: review, dtype: object

## Remove all HTML tags , URLs , Special Characters , & Stop words 

In [18]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Download the stopwords and print them
stop_words = stopwords.words("english")
print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SamaG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
def preprocess_text(text):
    if isinstance(text, str):
        # Remove any URLs or email addresses
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        text = re.sub(r'\S+@\S+', '', text)

        # Remove any sequence of alphanumeric characters starting with 'URL'
        text = re.sub(r'URL\w+', '', text)

        # Remove any non-alphanumeric characters
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Split the text into sentences
        sentences = sent_tokenize(text)

        preprocessed_sentences = []
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)

            # Remove any word contains numbers
            words = [word for word in words if not re.match('\w*\d\w*', word)]

            # Remove any word that is a stop word except "not", "Not", "IT", and "it"
            stop_words = set(nltk.corpus.stopwords.words('english'))
            stop_words.discard('not')
            stop_words.discard('Not')
            stop_words.discard('it')
            stop_words.discard('IT')
            words = [word for word in words if word.lower() not in stop_words or word.lower() in {'not', 'it'}]

            preprocessed_sentence = ' '.join(words)
            preprocessed_sentences.append(preprocessed_sentence)

        # Join the preprocessed sentences back together into a single string
        cleaned_text = ' '.join(preprocessed_sentences)

        return cleaned_text
    else:
        return text


In [20]:
dfr= dfr.apply(preprocess_text)

## Handling extra whitespace 

In [21]:
def remove_extra_whitespace(text):
    if isinstance(text, str):
        # Replace one or more whitespace characters with a single space
        text = re.sub(r'\s+', ' ', text)

        return text.strip()  # Remove leading and trailing whitespace
    else:
        return text

In [22]:
dfr = dfr.apply(remove_extra_whitespace)

In [23]:
dfr.describe()

count                                                 50000
unique                                                49579
top       loved todays show it variety not solely cookin...
freq                                                      5
Name: review, dtype: object

In [24]:
dfr2 = pd.DataFrame(data=dfr.values, columns=['reviews'])

In [25]:
dfr2

,reviews
0,one reviewers mentioned watching oz episode yo...
1,wonderful little production br br filming tech...
2,thought wonderful way spend time hot summer we...
3,basically theres family little boy jake thinks...
4,petter matteis love time money visually stunni...
...,...
49995,thought movie right good job it wasnt creative...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,im going disagree previous comment side maltin...


In [26]:
#words = word_tokenize(dfr)
dfr2['word_tokens'] = dfr2['reviews'].apply(word_tokenize)
dfr2

,reviews,word_tokens
0,one reviewers mentioned watching oz episode yo...,"[one, reviewers, mentioned, watching, oz, epis..."
1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,thought wonderful way spend time hot summer we...,"[thought, wonderful, way, spend, time, hot, su..."
3,basically theres family little boy jake thinks...,"[basically, theres, family, little, boy, jake,..."
4,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."
...,...,...
49995,thought movie right good job it wasnt creative...,"[thought, movie, right, good, job, it, wasnt, ..."
49996,bad plot bad dialogue bad acting idiotic direc...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,catholic taught parochial elementary schools n...,"[catholic, taught, parochial, elementary, scho..."
49998,im going disagree previous comment side maltin...,"[im, going, disagree, previous, comment, side,..."


## Expand contractions to their full forms

In [27]:
contractions_dict = {
    "can't": "cannot",
    "won't": "will not",
    "n't": " not",
    "'s": " is",
    "'re": " are",
    "'d": " would",
    "'ll": " will",
    "'t": " not",
    "'ve": " have"
}

In [28]:
def expand_contractions(text_series, contractions_dict):
    expanded_text = text_series.apply(lambda x: ' '.join([contractions_dict[word] if word in contractions_dict else word for word in x]))
    return expanded_text

In [29]:
expanded_text = expand_contractions(dfr2['word_tokens'], contractions_dict)
expanded_text


0        one reviewers mentioned watching oz episode yo...
1        wonderful little production br br filming tech...
2        thought wonderful way spend time hot summer we...
3        basically theres family little boy jake thinks...
4        petter matteis love time money visually stunni...
                               ...                        
49995    thought movie right good job it wasnt creative...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary schools n...
49998    im going disagree previous comment side maltin...
49999    one expects star trek movies high art fans exp...
Name: word_tokens, Length: 50000, dtype: object

In [30]:
dfr2

,reviews,word_tokens
0,one reviewers mentioned watching oz episode yo...,"[one, reviewers, mentioned, watching, oz, epis..."
1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,thought wonderful way spend time hot summer we...,"[thought, wonderful, way, spend, time, hot, su..."
3,basically theres family little boy jake thinks...,"[basically, theres, family, little, boy, jake,..."
4,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."
...,...,...
49995,thought movie right good job it wasnt creative...,"[thought, movie, right, good, job, it, wasnt, ..."
49996,bad plot bad dialogue bad acting idiotic direc...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,catholic taught parochial elementary schools n...,"[catholic, taught, parochial, elementary, scho..."
49998,im going disagree previous comment side maltin...,"[im, going, disagree, previous, comment, side,..."


## Remove duplicates (except "not")

In [31]:
def remove_duplicates(text):
    if isinstance(text, list):
        word_list = text
    elif isinstance(text, str):
        word_list = text.split()
    else:
        return text
    
    unique_words = []
    for i, word in enumerate(word_list):
        if word not in unique_words:
            unique_words.append(word)
        elif i == 0:
            continue
        else:
            unique_words[-1] += ' ' + word
    
    if isinstance(text, list):
        return unique_words
    elif isinstance(text, str):
        return ' '.join(unique_words)


In [32]:
dfr2['reviews'] = dfr2['reviews'].apply(remove_duplicates)

In [33]:
dfr2

,reviews,word_tokens
0,one reviewers mentioned watching oz episode yo...,"[one, reviewers, mentioned, watching, oz, epis..."
1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,thought wonderful way spend time hot summer we...,"[thought, wonderful, way, spend, time, hot, su..."
3,basically theres family little boy jake thinks...,"[basically, theres, family, little, boy, jake,..."
4,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."
...,...,...
49995,thought movie right good job it wasnt creative...,"[thought, movie, right, good, job, it, wasnt, ..."
49996,bad plot bad dialogue bad acting idiotic direc...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,catholic taught parochial elementary schools n...,"[catholic, taught, parochial, elementary, scho..."
49998,im going disagree previous comment side maltin...,"[im, going, disagree, previous, comment, side,..."


## Lemmatization

In [34]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    if isinstance(text, str):
        # Split the text into sentences
        sentences = sent_tokenize(text)

        # Lemmatize each word in each sentence
        lemmatized_sentences = []
        for sentence in sentences:
            words = nltk.word_tokenize(sentence)
            lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
            lemmatized_sentences.append(' '.join(lemmatized_words))

        # Join the lemmatized sentences back together into a single string
        lemmatized_text = ' '.join(lemmatized_sentences)
        return lemmatized_text
    else:
        return text

In [35]:
dfr2['reviews'] = dfr2['reviews'].apply(lemmatize_text)

In [36]:
dfr2

,reviews,word_tokens
0,one reviewer mentioned watching oz episode you...,"[one, reviewers, mentioned, watching, oz, epis..."
1,wonderful little production br br filming tech...,"[wonderful, little, production, br, br, filmin..."
2,thought wonderful way spend time hot summer we...,"[thought, wonderful, way, spend, time, hot, su..."
3,basically there family little boy jake think t...,"[basically, theres, family, little, boy, jake,..."
4,petter matteis love time money visually stunni...,"[petter, matteis, love, time, money, visually,..."
...,...,...
49995,thought movie right good job it wasnt creative...,"[thought, movie, right, good, job, it, wasnt, ..."
49996,bad plot bad dialogue bad acting idiotic direc...,"[bad, plot, bad, dialogue, bad, acting, idioti..."
49997,catholic taught parochial elementary school nu...,"[catholic, taught, parochial, elementary, scho..."
49998,im going disagree previous comment side maltin...,"[im, going, disagree, previous, comment, side,..."


## Vectorization

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features=1000)

#We fit and transform the column of the data frame using the fit_transform() method of the count vectorizer. This creates a sparse matrix of term frequencies for each document in the corpus
# fit and transform the using CountVectorizer
bag_of_words = count_vectorizer.fit_transform(dfr2['reviews'])

# get the feature names (unique words) from CountVectorizer
feature_names = count_vectorizer.get_feature_names_out()
# print(feature_names)
# create a new data frame using the bag of words matrix and feature names
bow_df = pd.DataFrame(bag_of_words.toarray(), columns=feature_names)

# print the resulting bag of words data frame
print(bow_df)

       ability  able  absolutely  accent  across  act  acted  acting  action  \
0            0     0           0       0       0    0      0       0       0   
1            0     0           0       0       0    0      0       0       0   
2            0     0           0       0       0    0      0       0       0   
3            0     0           0       0       0    0      0       0       0   
4            0     0           0       0       0    0      0       1       1   
...        ...   ...         ...     ...     ...  ...    ...     ...     ...   
49995        0     0           0       0       0    0      0       0       0   
49996        0     0           0       0       0    0      0       1       0   
49997        0     0           0       0       0    1      0       1       0   
49998        0     0           0       0       1    0      0       0       0   
49999        0     0           0       0       0    0      0       0       0   

       actor  ...  year  yes  yet  york

In [38]:
# Create a TF-IDF vectorizer object
tfidf_vectorizer = TfidfVectorizer(max_features=1000)

# Fit the vectorizer object on the text column
X_train_tfidf = tfidf_vectorizer.fit_transform(dfr2['reviews'])

# # Transform the text column into a sparse matrix of TF-IDF features
# tfidf_features = tfidf_vectorizer.transform(df['all_info'])

# Print the feature matrix shape
print('TF-IDF feature matrix shape:', X_train_tfidf.shape)
tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=feature_names)

print(tfidf_df)
print(tfidf_df.shape)

TF-IDF feature matrix shape: (50000, 1000)
       ability  able  absolutely  accent    across       act  acted    acting  \
0          0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.000000   
1          0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.000000   
2          0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.000000   
3          0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.000000   
4          0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.067468   
...        ...   ...         ...     ...       ...       ...    ...       ...   
49995      0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.000000   
49996      0.0   0.0         0.0     0.0  0.000000  0.000000    0.0  0.123777   
49997      0.0   0.0         0.0     0.0  0.000000  0.115781    0.0  0.075652   
49998      0.0   0.0         0.0     0.0  0.160979  0.000000    0.0  0.000000   
49999      0.0   0.0         0.0     0.0  0.000000  0.000000    0.

In [39]:
tfidf_df

,ability,able,absolutely,accent,across,act,acted,acting,action,actor,...,year,yes,yet,york,youll,young,younger,youre,youve,zombie
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.091298,0.00000,0.0,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.092043,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.102416,0.0,0.0,0.000000,0.000000,0.12382,0.0,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.121704,0.0,0.169975
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.067468,0.088739,0.000000,...,0.000000,0.0,0.0,0.127001,0.000000,0.00000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000
49996,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.123777,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000
49997,0.0,0.0,0.0,0.0,0.000000,0.115781,0.0,0.075652,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000
49998,0.0,0.0,0.0,0.0,0.160979,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000


In [40]:
tfidf_df.shape

(50000, 1000)

In [41]:
dfs.reshape(50000,1)

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [42]:
dfs.shape

(50000,)

## TRAIN-TEST SPLITTING

In [43]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, dfs, test_size=0.25, random_state=101)

In [46]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(bow_df, dfs, test_size=0.25, random_state=101)

## Models 

### logistic regression

In [47]:

#Bag of words
classifier = LogisticRegression(penalty='l2',solver='lbfgs',multi_class='ovr',class_weight='balanced')
classifier.fit(X_train2 , y_train2)
prediction2=classifier.predict(X_test2)
#model_score = accuracy_score(y_train, prediction)
model_score = accuracy_score(y_test2, prediction2)
class_report=classification_report(y_test2, prediction2)
# Print the accuracy report, scores and confusion matrix
print("Logistic Regression using all the features:")
print("Accuracy:","\n", model_score)
print("Accuracy report:","\n", class_report)
print("Confusion Matrix:","\n", confusion_matrix(y_test, prediction))

Logistic Regression using all the features:
Accuracy: 
 0.8588
Accuracy report: 
               precision    recall  f1-score   support

           0       0.86      0.85      0.86      6217
           1       0.86      0.87      0.86      6283

    accuracy                           0.86     12500
   macro avg       0.86      0.86      0.86     12500
weighted avg       0.86      0.86      0.86     12500

Confusion Matrix: 
 [[5322  895]
 [ 846 5437]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [45]:

#TFIDF
classifier = LogisticRegression(penalty='l2',solver='lbfgs',multi_class='ovr',class_weight='balanced')
classifier.fit(X_train , y_train)
prediction=classifier.predict(X_test)
#model_score = accuracy_score(y_train, prediction)
model_score = accuracy_score(y_test, prediction)
class_report=classification_report(y_test, prediction)
# Print the accuracy report, scores and confusion matrix
print("Logistic Regression using all the features:")
print("Accuracy:","\n", model_score)
print("Accuracy report:","\n", class_report)
print("Confusion Matrix:","\n", confusion_matrix(y_test, prediction))

Logistic Regression using all the features:
Accuracy: 
 0.86072
Accuracy report: 
               precision    recall  f1-score   support

           0       0.86      0.86      0.86      6217
           1       0.86      0.87      0.86      6283

    accuracy                           0.86     12500
   macro avg       0.86      0.86      0.86     12500
weighted avg       0.86      0.86      0.86     12500

Confusion Matrix: 
 [[5322  895]
 [ 846 5437]]


### Random Forest

In [51]:
# TFIDF
rfc = RandomForestClassifier(n_estimators=150, max_depth=15, min_samples_split=10)


rfc.fit(X_train, y_train)


y_pred = rfc.predict(X_test)

conf_mat = confusion_matrix(y_test,y_pred)

print("accuracy of RF': {}".format(accuracy_score(y_pred,y_test)))
print(conf_mat)
print(metrics.classification_report(y_test,y_pred))

accuracy of RF': 0.8164
[[4801 1416]
 [ 879 5404]]
              precision    recall  f1-score   support

           0       0.85      0.77      0.81      6217
           1       0.79      0.86      0.82      6283

    accuracy                           0.82     12500
   macro avg       0.82      0.82      0.82     12500
weighted avg       0.82      0.82      0.82     12500



In [52]:
# Bag of words
rfc = RandomForestClassifier(n_estimators=150, max_depth=15, min_samples_split=10)


rfc.fit(X_train2, y_train2)


y_pred2 = rfc.predict(X_test2)

conf_mat = confusion_matrix(y_test2,y_pred2)

print("accuracy of RF': {}".format(accuracy_score(y_pred2,y_test2)))
print(conf_mat)
print(metrics.classification_report(y_test2,y_pred2))

accuracy of RF': 0.8116
[[4819 1398]
 [ 957 5326]]
              precision    recall  f1-score   support

           0       0.83      0.78      0.80      6217
           1       0.79      0.85      0.82      6283

    accuracy                           0.81     12500
   macro avg       0.81      0.81      0.81     12500
weighted avg       0.81      0.81      0.81     12500

